# Saved States Overview

In [60]:
from core import game

# save_dir = "saves/ai_battles/ai_battle_028"
save_dir = "saves/llama3.2_1b_single/llama3.2_1b_single/round_013"

In [23]:
import os
pkl_files = []
for file in os.listdir(save_dir):
    if file.endswith(".pkl"):
        pkl_files.append(file)

print("Found pkl files:", pkl_files)

Found pkl files: ['turn_005.pkl', 'turn_000.pkl', 'latest.pkl', 'turn_006.pkl', 'turn_007.pkl', 'turn_003.pkl']


In [66]:
import pickle
import json
# select_files = pkl_files.copy()
# select_files = ["turn_002.pkl"]
save_dir = ""
select_files = ['saves/llama3.2_3b_single/llama3.1_8b_single/round_002/turn_014.pkl']
for file in select_files:
    with open(os.path.join(save_dir, file), "rb") as f:
        state = pickle.load(f)
        print(f"=== Loaded state from {file}: ===")
        print(f"Keys:")
        for k, v in state.items():
            if type(v) is dict:
                print(f"  {k}: {[f'{kk}: {type(vv).__name__}' for kk, vv in v.items()]} | {len(v)} items")
            elif type(v) is list and len(v) > 0 and type(v[-1]) is dict:
                print(f"  {k}: list({[f'{kk}: {type(vv).__name__}' for kk, vv in v[-1].items()]}) | {len(v)} items")
        print(json.dumps(state, indent=2))


=== Loaded state from saves/llama3.2_3b_single/llama3.1_8b_single/round_002/turn_014.pkl: ===
Keys:
  state: ['players: list', 'round_no: int', 'turn_player: int', 'move_history: list', 'game_turn: int', 'player_turn: int', 'phase: str', 'current_player: int', 'max_tries: int'] | 9 items
  history: list(['player: int', 'game_turn: int', 'player_turn: int', 'input_messages: list', 'response: dict', 'valid: bool', 'move: str', 'damage_dealt: int', 'damage_taken: int', 'attempt: int', 'max_attempts: int']) | 14 items
  evaluation: ['players: dict', 'game_status: str'] | 2 items
  players: ['1: list', '2: list'] | 2 items
{
  "state": {
    "players": [
      {
        "name": "AI Alpha",
        "roster": [
          "Electric Eel",
          "Sunfish",
          "Sea Wolf",
          "Sea Turtle",
          "Octopus",
          "Hammerhead Shark",
          "Clownfish",
          "Mimic Fish"
        ],
        "team": {
          "fish": [
            {
              "name": "Archerfish

# Inference testing

In [13]:
from langchain_ollama import ChatOllama
model = "gpt-oss:20b"
temperature = 0.1
top_p = 0.9

llm = ChatOllama(
    model=model,
    temperature=temperature,
    top_p=top_p,
)
response = llm.invoke("What is the capital of France?")
print("LLM response:", response)

KeyboardInterrupt: 

In [40]:
import json
res_json = json.loads(response.model_dump_json())
print(json.dumps(res_json, indent=2))

{
  "content": "The capital of France is **Paris**.",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "gpt-oss:20b",
    "created_at": "2025-08-15T18:18:15.029567396Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 90705936382,
    "load_duration": 89579361336,
    "prompt_eval_count": 74,
    "prompt_eval_duration": 577526484,
    "eval_count": 40,
    "eval_duration": 548376871,
    "model_name": "gpt-oss:20b"
  },
  "type": "ai",
  "name": null,
  "id": "run--f33e3fa7-0680-4afc-9776-3f3a5d116d0e-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 74,
    "output_tokens": 40,
    "total_tokens": 114
  }
}
